In [ ]:
import requests

user_agent_url = 'https://www.chicagofed.org/~/media/others/banking/financial-institution-reports/merger-files/bankmergerdata-xml.xml?la=en'
xml_data_banks = requests.get(user_agent_url).content

user_agent_url = 'https://www.chicagofed.org/~/media/others/banking/financial-institution-reports/merger-files/bhcmergerdata-xml.xml?la=en'
xml_data_bhc = requests.get(user_agent_url).content

In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
class XML2DataFrame:

    def __init__(self, xml_data):
        self.root = ET.XML(xml_data)

    def parse_root(self, root):
        return [self.parse_element(child) for child in iter(root)]

    def parse_element(self, element, parsed=None):
        if parsed is None:
            parsed = dict()
        for key in element.keys():
            parsed[key] = element.attrib.get(key)
        if element.text:
            parsed[element.tag] = element.text
        for child in list(element):
            self.parse_element(child, parsed)
        return parsed

    def process_data(self):
        structure_data = self.parse_root(self.root)
        return pd.DataFrame(structure_data)

In [ ]:
xml2df_banks = XML2DataFrame(xml_data_banks)
xml_df_banks = xml2df_banks.process_data()

xml2df_bhc = XML2DataFrame(xml_data_bhc)
xml_df_bhc = xml2df_bhc.process_data()

In [ ]:
xml_df_banks.head()

In [ ]:
xml_df_banks_merge = xml_df_banks.drop(['BANK_MERGER'], axis=1).copy()
xml_df_banks_merge['Merger_Type'] = 'Bank'

In [ ]:
xml_df_banks_merge.head()

In [ ]:
xml_df_bhc.head()

In [ ]:
xml_df_bhc_merge = xml_df_bhc.drop(['BHC_MERGER'], axis=1).copy()
xml_df_bhc_merge['Merger_Type'] = 'BHC'

In [ ]:
xml_df_bhc_merge.head()

In [ ]:
df_bhc_banks = pd.concat([xml_df_banks_merge, xml_df_bhc_merge])

In [ ]:
df_bhc_banks.head()

In [ ]:
df_bhc_banks['merge_year'] = df_bhc_banks.apply(lambda x: x['MERGE_DT'][:5], axis=1)
df_bhc_banks['merge_month'] = df_bhc_banks.apply(lambda x: x['MERGE_DT'][5:7], axis=1)
print(df_bhc_banks['merge_year'][0])
print(df_bhc_banks['merge_month'][0])

In [ ]:
df_bhc_banks.to_csv('bank_bhc_merger_data.csv')

In [ ]:
grouped_by_year = df_bhc_banks.groupby(['merge_year']).size().reset_index()
grouped_by_year

In [ ]:
grouped_by_survivor = df_bhc_banks.groupby(['SURV_NM']).size().reset_index().copy()
grouped_by_survivor.sort_values(by=[0], ascending=False)[:50]

In [ ]:
grouped_by_non_survivor = df_bhc_banks.groupby(['NON_SURV']).size().reset_index().copy()
grouped_by_non_survivor.sort_values(by=[0], ascending=False)[:50]

In [ ]:
trillion_banks_list = []
trillion_mapping = {}
for i in grouped_by_survivor['SURV_NM']:
    if "JP" in i:
        trillion_banks_list.append(i)
        trillion_mapping[i] = 'JP MORGAN CHASE'
    elif "WELLS FARGO" in i:
        trillion_banks_list.append(i)
        trillion_mapping[i] = 'WELLS FARGO'
    elif "CITIBANK" in i:
        trillion_banks_list.append(i)
        trillion_mapping[i] = 'CITIBANK'
    elif "CITI-BANCSHARES" in i:
        trillion_banks_list.append(i)
        trillion_mapping[i] = 'CITIBANK'
    elif "BANK OF AMERICA" in i:
        trillion_banks_list.append(i)
        trillion_mapping[i] = 'BANK OF AMERICA'
    elif "BANKAMERICA" in i:
        trillion_banks_list.append(i)
        trillion_mapping[i] = 'BANK OF AMERICA'
    elif "NATIONSBANK" in i:
        trillion_banks_list.append(i)
        trillion_mapping[i] = 'BANK OF AMERICA'

In [ ]:
for i in grouped_by_non_survivor['NON_SURV']:
    if "JP" in i:
        trillion_banks_list.append(i)
    elif "WELLS FARGO" in i:
        trillion_banks_list.append(i)
    elif "CITIBANK" in i:
        trillion_banks_list.append(i)
    elif "CITI-BANCSHARES" in i:
        trillion_banks_list.append(i)
    elif "BANK OF AMERICA" in i:
        trillion_banks_list.append(i)
    elif "BANKAMERICA" in i:
        trillion_banks_list.append(i)
    elif "NATIONSBANK" in i:
        trillion_banks_list.append(i)

In [ ]:
trillion_banks_list

In [ ]:
subset_df = df_bhc_banks[df_bhc_banks['SURV_NM'].isin(trillion_banks_list)]
subset_df

In [ ]:
subset_df = subset_df[~subset_df['NON_SURV'].isin(trillion_banks_list)].copy()
subset_df

In [ ]:
subset_df['NEW_SURV_NM'] = subset_df['SURV_NM'].map(trillion_mapping)

In [ ]:
subset_df

In [ ]:
subset_df = subset_df.sort_values(by=['MERGE_DT']).reset_index()

In [ ]:
subset_df

In [ ]:
len(subset_df)

In [ ]:
subset_df.columns

In [ ]:
import networkx as nx

In [ ]:
G = nx.Graph()

In [ ]:
# for i in range(len(df_bhc_banks)):
list_of_non_surviving_banks = []
for i in range(len(subset_df)):
    non_surviving = subset_df['NON_SURV'][i]
    list_of_non_surviving_banks.append(non_surviving)
    surviving = subset_df['NEW_SURV_NM'][i]
    G.add_edge(non_surviving, surviving)

In [ ]:
list_of_non_surviving_banks

In [ ]:
color_list = []
for i in list(G.nodes):
    if i in list_of_non_surviving_banks:
        color_list.append('red')
    else:
        color_list.append('green')

In [ ]:
print(len(color_list))
print(len(list_of_non_surviving_banks))
print(len(list(G.nodes)))

In [ ]:
from bokeh.io import show, output_file
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, TapTool, BoxSelectTool, WheelZoomTool, PanTool, ResetTool
from bokeh.models.graphs import from_networkx, NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.palettes import Spectral4

In [ ]:
plot = Plot(plot_width=800, plot_height=800,
            x_range=Range1d(-1.1,1.1), y_range=Range1d(-1.1,1.1))
plot.title.text = "Graph Interaction Demonstration"



hover = HoverTool(tooltips=[("Bank:", "@name")])

plot.add_tools(hover, WheelZoomTool(), PanTool(), ResetTool())

graph_renderer = from_networkx(G, nx.spring_layout, scale=2, center=(0,0))

graph_renderer.selection_policy = NodesAndLinkedEdges()
graph_renderer.inspection_policy = NodesAndLinkedEdges()

graph_renderer.node_renderer.data_source.data['name'] = list(G.nodes)
graph_renderer.node_renderer.data_source.data['node_color'] = color_list




In [ ]:
graph_renderer.node_renderer.data_source.data

In [ ]:
graph_renderer.node_renderer.hover_glyph = Circle(size=7, fill_color=Spectral4[1])
graph_renderer.node_renderer.glyph = Circle(size=7, fill_color='node_color')

graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color=Spectral4[1], line_width=5)

plot.renderers.append(graph_renderer)

output_file("interactive_graphs.html")
show(plot)

In [ ]:
graph.node_renderer

In [ ]:
from bokeh.io import show, output_file
from bokeh.plotting import figure
from bokeh.models.graphs import from_networkx

In [ ]:
# hover = HoverTool(tooltips=[("Name:", "@name")])

plot = figure(title="Networkx Integration Demonstration", x_range=(-1.1,1.1), y_range=(-1.1,1.1),
              tools="pan,wheel_zoom,box_zoom,reset,hover", toolbar_location='below')


graph = from_networkx(G, nx.spring_layout, scale=2, center=(0,0))
plot.renderers.append(graph)

output_file("networkx_graph.html")
show(plot)

In [ ]:
plot.renderers.

In [ ]:
G = nx.Graph()
G.add_edge('A', 'B')
# >>> G.add_edge('B', 'D', weight=2)
# >>> G.add_edge('A', 'C', weight=3)
# >>> G.add_edge('C', 'D', weight=4)
# >>> nx.shortest_path(G, 'A', 'D', weight='weight')
# ['A', 'B', 'D']

In [ ]:
fig_size[0] = 18
fig_size[1] = 9
plt.rcParams["figure.figsize"] = fig_size

In [ ]:
plt.plot(grouped_by_year['merge_year'], grouped_by_year[0])
plt.title('Bank Mergers')
plt.ylabel('Number of Mergers')
plt.xlabel('Merger by Year 1976-2015')
plt.show()

In [ ]:
fig = plt.figure(figsize=(4,3))
ax = fig.add_subplot(111)
ax.plot(list1, list2)
fig.savefig('fig1.png', dpi = 300)
fig.close()

In [ ]:
apply(lambda x: list(np.around(np.array(x[i])/x['segment_size'], decimals = 4)), axis = 1)

In [ ]:
def get_base_pvol_assumption(num_statments = 72):
    stmt_array = np.array(range(1,num_statments+1))
    pvol_array = 0.0457*(stmt_array**3)-3.2447*(stmt_array**2)+66.568*(stmt_array)+180.61
    for i in range(32,72):
        pvol_array[i] = pvol_array[31]
    return pvol_array
get_base_pvol_assumption()

In [ ]:
get_base_pvol_assumption()[:12]

In [ ]:
plt.plot(get_base_pvol_assumption())
plt.title('Base Purchase Volume/Open Assumption')
plt.ylabel('Purchase Volume')
plt.xlabel('Statment')
plt.show()

In [ ]:
def get_pvol_adjustment_factors():
    risk_array = np.array(range(10))
    usage_array = np.array(range(10))
    usage_adj_array = 1.0+(usage_array*.1)
    risk_adj_array = 1.0+(3-usage_array)*0.02
    segment_level_pvol_adj = np.multiply(usage_adj_array,risk_adj_array.reshape((10,1)))
    return usage_adj_array, risk_adj_array, segment_level_pvol_adj
get_pvol_adjustment_factors()

In [ ]:
def get_final_segment_level_pvols(base_pvol_assumptions, pvol_adjustment_factors):
    segment_pvol_curves = np.multiply(pvol_adjustment_factors.reshape(100,1),base_pvol_assumptions)
    return segment_pvol_curves
get_final_segment_level_pvols(get_base_pvol_assumption(), get_pvol_adjustment_factors()[2])

In [ ]:
get_final_segment_level_pvols(get_base_pvol_assumption(), get_pvol_adjustment_factors()[2])[2]

In [ ]:
for i in range(100):
    plt.plot(get_final_segment_level_pvols(get_base_pvol_assumption(get_stmt_array()), get_pvol_adjustment_factors()[2])[i])
plt.show()

In [ ]:
def get_segment_id_array():
    risk_usage_list
    for i in range(10):
        for j in range(10):
            risk_usage_list.append([i,j])
    segment_ids_risk_usage_array = np.array(risk_usage_list)
    return segment_ids_risk_usage_array
get_segment_id_array()

In [ ]:
base_line_array = np.zeros((10,10))

In [ ]:
base_line_array[0] = [7000.00, 8000.00, 9000.00, 10000.00, 
                                         12000.00, 15000.00, 17000.00, 18000.00,
                                         20000.00, 20000.00]
base_line_array[0]

In [ ]:
base_line_array[1] = np.around(base_line_array[0]*0.90,decimals=-2)
base_line_array[1]

In [ ]:
base_line_array.reshape(100,1)

In [ ]:
base_line_array = np.array([7000.00, 8000.00, 9000.00, 10000.00, 
                                         12000.00, 15000.00, 17000.00, 18000.00,
                                         20000.00, 20000.00])

In [ ]:
base_line_array[1]=base_line_array*0.92

In [ ]:
base_line_array = np.zeros((10,10))
base_line_array[0] = [7000.00, 8000.00, 9000.00, 10000.00,
                                   12000.00, 15000.00, 17000.00, 18000.00,
                                   20000.00, 20000.00]
for i in range(1,10):
    base_line_array[i] = np.around(base_line_array[i-1]*0.90,decimals=-2)
base_line_array = base_line_array.reshape(100,1)
(-0.00004)*base_line_array+1.2

In [ ]:
for i in base_line_array:
    print(i)

In [ ]:
risk_array = np.array(range(10))
risk_adj_array = np.around(1-(0.02*(4-risk_array)), decimals=2)
risk_adj_array

In [ ]:
usage_adj_array = np.array([1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 
                                    1.00, 1.00, 1.00, 1.00])
usage_adj_array

In [ ]:
segment_level_severity_adj = np.around(np.multiply(usage_adj_array, risk_adj_array.reshape((10,1))), decimals=4)
segment_level_severity_adj.reshape(100,1)

In [ ]:
import pandas as pd
import random 

In [ ]:
d = {'col1': [1, 2, 3, 4], 'col2': ['a','b','c', 'd']}
df = pd.DataFrame(data=d)

In [ ]:
df['col1']

In [ ]:
def sample_and_assign_TS(df):
    if df[df['col1'] <= 2].sample(n=1)
    return df_2

In [ ]:
test(df)

In [ ]:
import tarfile

In [ ]:
june_2016 = tarfile.open(name='test_june_2016.tar.gz')

In [ ]:
june_2016.extractall()

In [ ]:
acs_4k = tarfile.open(name='test_sample_acs_4k.tar.gz')

In [ ]:
acs_4k.extractall()

In [ ]:
coin_values = [1,2,5,10,20,50,100,200]

In [ ]:
max_of_each_value = {}
for i in coin_values:
    max_of_each_value[i] = 200/i

In [ ]:
max_of_each_value

In [ ]:
200/4

In [ ]:
import numpy as np
examp = [50,50,50,50]
for i in examp:
    if i>100 and i%100==0:
        examp.remove(i)
        addition = list(np.ones(int(i/100))*100)
        examp = examp + addition
        break
    elif i>50 and i%50==0:
        examp.remove(i)
        addition = list(np.ones(int(i/50))*50)
        examp = examp + addition
        break
    elif i>20 and i%20==0:
        examp.remove(i)
        addition = list(np.ones(int(i/20))*20)
        examp = examp + addition
        break
    elif i>10 and i%10==0:
        examp.remove(i)
        addition = list(np.ones(int(i/10))*10)
        examp = examp + addition
        break
    elif i>5 and i%5==0:
        examp.remove(i)
        addition = list(np.ones(int(i/5))*5)
        examp = examp + addition
        break
    elif i>2 and i%2==0:
        examp.remove(i)
        addition = list(np.ones(int(i/2))*2)
        examp = examp + addition
        break
    elif i>1 and i%1==0:
        examp.remove(i)
        addition = list(np.ones(int(i/2))*2)
        examp = examp + addition
        break

In [ ]:
def collect_options(coins):
    if len(coins) >= 200:
        print('All Options Collected')
    else:
        for i in coins:
            if i>100 and i%100==0:
                coins.remove(i)
                addition = list(np.ones(int(i/100))*100)
                new_coins = coins + addition
                options.append(new_coins)
                return collect_options(new_coins)
            elif i>50 and i%50==0:
                coins.remove(i)
                addition = list(np.ones(int(i/50))*50)
                new_coins = coins + addition
                options.append(new_coins)
                return collect_options(new_coins)
            elif i>20 and i%20==0:
                coins.remove(i)
                addition = list(np.ones(int(i/20))*20)
                new_coins = coins + addition
                options.append(new_coins)
                return collect_options(new_coins)
            elif i>10 and i%10==0:
                coins.remove(i)
                addition = list(np.ones(int(i/10))*10)
                new_coins = coins + addition
                options.append(new_coins)
                return collect_options(new_coins)
            elif i>5 and i%5==0:
                coins.remove(i)
                addition = list(np.ones(int(i/5))*5)
                new_coins = coins + addition
                options.append(new_coins)
                return collect_options(new_coins)
            elif i>2 and i%2==0:
                coins.remove(i)
                addition = list(np.ones(int(i/2))*2)
                new_coins = coins + addition
                options.append(new_coins)
                return collect_options(new_coins)
            elif i>1 and i%1==0:
                coins.remove(i)
                addition = list(np.ones(int(i/1))*1)
                new_coins = coins + addition
                options.append(new_coins)
                return collect_options(new_coins)

In [ ]:
coins = [200]
options = [[200.0]]
collect_options(coins)

In [ ]:
len(options)

In [ ]:
for i in options:
    print(i)

In [ ]:
test = [[200.0]]
test.append([100.0,100.0])

In [ ]:
test

In [ ]:
!ls

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [ ]:
driver = webdriver.Chrome()

In [ ]:
driver.get('https://www.plex.tv/sign-in/?forward=https://www.plex.tv/')

In [ ]:
elem = driver.find_elements_by_xpath("//*[contains(text(), 'email')]")[0]

In [ ]:
elem.click()

In [ ]:
email = driver.find_element_by_id("email")
password = driver.find_element_by_id("password")

In [ ]:
email.send_keys("tphoran@gmail.com")

In [ ]:
password.send_keys("VBiLfhU9dzVQJ")

In [ ]:
sign_in = driver.find_elements_by_xpath("//*[contains(text(), 'Sign in')]")[0]

In [ ]:
sign_in.click()

In [ ]:
email = 

In [ ]:
elem = driver.find_element_by_name('j_username')
elem.clear()
elem.send_keys('username')

In [ ]:
driver.find_element_by_css_selector('button._17iUOU._1I3Olm._1A8EcL._2kT68l._2n0yJn._3S9UdJ._1NBfEr._1l_5xB._1A8EcL._2kT68l._2n0yJn._2kT68l._2n0yJn._3fwLzo._3S9UdJ._2n0yJn._2XA2bN').click()

In [ ]:
_17iUOU._1I3Olm._1A8EcL._2kT68l._2n0yJn._3S9UdJ._1NBfEr._1l_5xB._1A8EcL._2kT68l._2n0yJn._2kT68l._2n0yJn._3fwLzo._3S9UdJ._2n0yJn._2XA2bN

In [ ]:
driver.find_element_by_id('email')

In [ ]:
driver.sendKeys(Keys.TAB);
driver.sendKeys(Keys.TAB);
driver.sendKeys(Keys.TAB);
driver.sendKeys(Keys.ENTER);

In [ ]:
<button data-qa-id="signIn--email" type="submit" role="button" class="_17iUOU _1I3Olm _1A8EcL _2kT68l _2n0yJn _3S9UdJ _1NBfEr _1l_5xB _1A8EcL _2kT68l _2n0yJn  _2kT68l _2n0yJn _3fwLzo _3S9UdJ _2n0yJn _2XA2bN      "><span class="_1NdZWc"><div class="_1TgDPI Niere7 _2kLwt_ _3PStHE Niere7 _2kLwt_" aria-label="Loading"></div></span><span class="qxG01S">Continue with email</span></button>

In [ ]:
import boto3
import awscli

In [ ]:
ec2 = boto3.resource('ec2', region_name="us-east-1")

In [ ]:
--tag-specifications "ResourceType=instance,Tags=[{Key=Owner,Value=TeamA}]"

In [ ]:
from imdb import IMDb
ia = IMDb()

In [ ]:
# Search for a movie (get a list of Movie objects).
s_result = ia.search_movie('The Untouchables')

In [ ]:
s_result[0]

In [ ]:
for item in s_result:
   print(item['long imdb canonical title'], item.movieID)

In [ ]:
the_unt = s_result[0]
ia.update(the_unt)

In [ ]:
print(ia.get_movie_infoset())

In [ ]:
the_unt.keys()

In [ ]:
the_unt['year']

In [ ]:
the_unt['plot']

In [1]:
import boto3
import datetime
from imdb import IMDb
from boto3.dynamodb.conditions import Key, Attr

In [2]:
today = datetime.datetime.today().strftime('%Y %m %d')

In [3]:
ia = IMDb()
client = boto3.client('s3')
results = client.list_objects(Bucket='manny-and-meche-plex', Prefix = 'Movies/')

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('plex-collection')

In [4]:
movie_list = []
for i in results['Contents']:
    movie = i['Key'][7:-4]
    if movie != '':
        movie_list.append(movie)

In [ ]:
manual_list = ['Avatar', 'Insomnia', 'A Christmas Carol', 'Star Trek', 'The Godfather: Part 2', 'The Godfather: Part 3', 'Angels and Demons']

In [ ]:
# movie_list = ['10 Cloverfield Lane']

In [ ]:
for m in movie_list:
    m_search = m.replace(" ", "").replace(":", "").replace("-", "").replace("'", "").replace(".", "").replace(",","")
    m_search = m_search.lower()
    if table.query(KeyConditionExpression=Key('Search Title').eq(m_search))['Count'] == 0:
        try:
            if m not in manual_list:
                s_result = ia.search_movie(m)
                entry = s_result[0]
                ia.update(entry)
                data_input = {}
                data_input['Search Title'] = m_search
                data_input['Title'] = m
                data_input['Plot'] = entry['plot'][0]
                data_input['Genres'] = entry['genres']
                release_date = entry['original air date'][:11]
                datetime_object = datetime.datetime.strptime(release_date, '%d %b %Y')
                data_input['Release Date'] = datetime_object.strftime('%Y %m %d')
                data_input['Add Date'] = today
                data_input['Year'] = datetime_object.strftime('%Y')
                table.put_item(Item=data_input)
        except (KeyError, ValueError) as e:
            print('Failed '+m)
    else:
        print(m+' already in database')

In [ ]:
for m in movie_list:
    if m not in manual_list:
        s_result = ia.search_movie(m)
        print(m)
        print(s_result[0])
        print()

In [5]:
# 'Avatar (2009)', 'Insomnia (2002)', 'A Christmas Carol (1984)', 'Star Trek (2009), 'Angels & Demons (2009)', \
# 'The Godfather: Part 2', 'The Godfather: Part 3', 'South Park: Bigger, Longer and Uncut'
s_result = ia.search_movie('Ant-Man and the Wasp')
entry = s_result[0]
ia.update(entry)
m = 'Ant-Man and the Wasp' 
m_search = m.replace(" ", "").replace(":", "").replace("-", "").replace("'", "").replace(".", "").replace(",","")
m_search = m_search.lower()
data_input = {}
data_input['Search Title'] = m_search
data_input['Title'] = m
data_input['Plot'] = entry['plot'][0]
data_input['Genres'] = entry['genres']
release_date = entry['original air date'][:11]
datetime_object = datetime.datetime.strptime(release_date, '%d %b %Y')
data_input['Release Date'] = datetime_object.strftime('%Y %m %d')
data_input['Add Date'] = today
data_input['Year'] = datetime_object.strftime('%Y')

In [6]:
data_input

{'Add Date': '2018 11 17',
 'Genres': ['Action', 'Adventure', 'Comedy', 'Sci-Fi'],
 'Plot': 'As Scott Lang balances being both a Super Hero and a father, Hope van Dyne and Dr. Hank Pym present an urgent new mission that finds the Ant-Man fighting alongside The Wasp to uncover secrets from their past.::Walt Disney Studios',
 'Release Date': '2018 07 06',
 'Search Title': 'antmanandthewasp',
 'Title': 'Ant-Man and the Wasp',
 'Year': '2018'}

In [9]:
table.put_item(Item=data_input)

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '2',
   'content-type': 'application/x-amz-json-1.0',
   'date': 'Mon, 05 Nov 2018 01:58:27 GMT',
   'server': 'Server',
   'x-amz-crc32': '2745614147',
   'x-amzn-requestid': 'R1UHRMU3AF0O6HLTO6LRG1P0QFVV4KQNSO5AEMVJF66Q9ASUAAJG'},
  'HTTPStatusCode': 200,
  'RequestId': 'R1UHRMU3AF0O6HLTO6LRG1P0QFVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'RetryAttempts': 0}}

In [ ]:
s_result

In [ ]:
s_result = ia.search_movie("Family Guy: Something, Something, Something, Dark Side")
entry = s_result[0]
ia.update(entry)
print(entry['original air date'][:11])
data = entry.data
# for i in data:
#     print(i)

In [ ]:
x = data['original air date'][:11]

In [ ]:
datetime_object = datetime.datetime.strptime(release_date, '%d %b %Y')

In [ ]:
datetime_object.strftime('%Y %m %d')

In [ ]:
movie = 'Ant Man'
dynamodb = boto3.resource('dynamodb')
movie_table = dynamodb.Table('plex-collection')
if movie_table.query(KeyConditionExpression=Key('Title').eq(movie))['Count'] > 0:
    print('fart')
else:
    words=movie.split()
    if words[0].lower() == 'the' or words[0].lower() == 'a':
        search_word = words[1]
    else: 
        search_word = words[0]
    search_word = search_word.split(':')[0]
    search_word = search_word.split('-')[0]
    search_results = movie_table.scan(Select='ALL_ATTRIBUTES', FilterExpression=Attr('Title').contains(search_word.capitalize()))
    if search_results['Count']<=15:
        results_to_return = search_results['Count']
    else:
        results_to_return = 15
    for i in range(results_to_return):
        print(search_results['Items'][i]['Title'])

In [ ]:
word = 'Ant-Man,'
word[:-1]

In [ ]:
search_results

In [ ]:
movie = 'ant man'
dynamodb = boto3.resource('dynamodb')
movie_table = dynamodb.Table('plex-collection')
response = movie_table.scan(Select='ALL_ATTRIBUTES', FilterExpression=Attr('Title').contains("Ant"))

In [ ]:
response

In [ ]:
for i in range(1):
    print(i)

In [ ]:
print('Star\nWars')

In [ ]:
dynamodb = boto3.resource('dynamodb')
movie_table = dynamodb.Table('plex-collection')

In [ ]:
response = movie_table.scan(Select='ALL_ATTRIBUTES', FilterExpression=Attr('Release Date').gt("2015"))

In [ ]:
response_dict = {}
for i in response['Items']:
    response_dict[i['Release Date']] = i['Title']        

In [ ]:
release_dates = list(response_dict.keys())

In [ ]:
release_dates.sort(reverse=True)
recent = ''
for i in release_dates[:10]:
    recent += response_dict[i]+', '

In [ ]:
recent

In [ ]:
for key in sorted(response_dict.iterkeys()):
    print "%s: %s" % (key, mydict[key])

In [ ]:
mydict = {'carl':40,
          'alan':2,
          'bob':1,
          'danny':3}

for key in sorted(mydict.iterkeys()):
    print("%s: %s" % (key, mydict[key]))

In [ ]:
for i in response_dict.values():
    print(i)

In [ ]:
'2018 04 06' > '2016 03 11'

In [ ]:
response_dict

In [ ]:
response = movie_table.query(Select='SPECIFIC_ATTRIBUTES',
                             AttributesToGet=['Title','Release Date'],
                             KeyConditionExpression=
                             ScanIndexForward=True)

In [ ]:
Select='ALL_ATTRIBUTES'|'ALL_PROJECTED_ATTRIBUTES'|'SPECIFIC_ATTRIBUTES'|'COUNT',
    AttributesToGet=[
        'string',
    ],

In [ ]:
response

In [ ]:
dynamodb = boto3.resource('dynamodb')
movie_table = dynamodb.Table('plex-collection')
response = movie_table.scan(Select='ALL_ATTRIBUTES', FilterExpression=Attr('Year').gt("2016"))
    
movie_table.query(KeyConditionExpression=Key('Search Title').eq('antman'))['Items'][0]['Title']

In [ ]:
recent

In [ ]:
response_dict['2018 01 05']

In [ ]:
response

In [ ]:
release_dates

In [ ]:
def happy_path():
    return 'test'

In [ ]:
def relaunch_if_needed():
    if happy_path() == 'test':
        return 'yay'

In [ ]:
happy_path()

In [ ]:
relaunch_if_needed()

In [ ]:
test_list = [1,2,3,4,4]
len(set(test_list))


In [1]:
import boto3

In [2]:
sns_client = boto3.client('sns')

In [10]:
sns_client.publish(PhoneNumber='+15712559339', Message='Message WooWoo')

{'MessageId': '70c78709-46fc-5a59-8a0e-d3ac3aecef56',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '294',
   'content-type': 'text/xml',
   'date': 'Tue, 01 Jan 2019 14:31:43 GMT',
   'x-amzn-requestid': '071a66f5-ec55-5f5c-8f2f-32509045aa53'},
  'HTTPStatusCode': 200,
  'RequestId': '071a66f5-ec55-5f5c-8f2f-32509045aa53',
  'RetryAttempts': 0}}